In [2]:
import pandas as pd

plays = pd.read_csv("~/nfl-data-bowl/Sample_Data/Raw/plays.csv")

thru_snap = pd.read_csv("~/nfl-data-bowl/Sample_Data/Raw/playersThroughSnap.csv")

thru_snap.describe()


,Game ID,Play ID,Moving Offensive Before Snap,Moving Defensive Before Snap,Moving Offensive After Snap,Moving Defensive After Snap
count,3.850000e+03,3850.000000,3850.000000,3850.000000,3850.000000,3850.000000
mean,2.022098e+09,2206.603896,4.045974,2.529351,5.959740,2.645195
std,4.889138e+03,1255.333700,1.750245,0.685595,0.221453,0.597330
min,2.022091e+09,54.000000,0.000000,0.000000,3.000000,0.000000
25%,2.022092e+09,1124.500000,3.000000,2.000000,6.000000,2.000000
50%,2.022100e+09,2185.500000,4.000000,3.000000,6.000000,3.000000
75%,2.022102e+09,3270.750000,6.000000,3.000000,6.000000,3.000000
max,2.022103e+09,5120.000000,6.000000,4.000000,6.000000,4.000000


In [3]:
plays2 = plays[["gameId", "playId","expectedPointsAdded","yardsGained"]]

thru_snap = thru_snap.rename(columns={'Game ID': 'gameId'})

thru_snap = thru_snap.rename(columns={'Play ID': 'playId'})

thru_snap = plays2.merge(thru_snap, on=['gameId', 'playId'], how='left')
thru_snap = thru_snap.fillna(0)

thru_snap = thru_snap.rename(columns={'expectedPointsAdded_x': 'expectedPointsAdded'})
thru_snap = thru_snap.rename(columns={'yardsGained_x': 'yardsGained'})

thru_snap['bigYardPlay'] = (thru_snap['yardsGained'] > 15).astype(int)
thru_snap['bigEpaPlay'] = (thru_snap['expectedPointsAdded'] > 1.5).astype(int)

thru_snap.describe()

,gameId,playId,expectedPointsAdded,yardsGained,Moving Offensive Before Snap,Moving Defensive Before Snap,Moving Offensive After Snap,Moving Defensive After Snap,bigYardPlay,bigEpaPlay
count,1.612400e+04,16124.000000,16124.000000,16124.000000,16124.000000,16124.000000,16124.000000,16124.000000,16124.000000,16124.000000
mean,2.022099e+09,2023.830563,-0.029381,5.460618,0.966075,0.603944,1.423034,0.631605,0.098859,0.110146
std,5.979252e+03,1182.090771,1.397405,8.833095,1.925333,1.129213,2.543225,1.164925,0.298482,0.313082
min,2.022091e+09,54.000000,-13.023600,-68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.022093e+09,996.000000,-0.645440,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.022101e+09,2017.000000,-0.178783,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.022102e+09,3022.000000,0.696452,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2.022111e+09,5120.000000,8.698986,98.000000,6.000000,4.000000,6.000000,4.000000,1.000000,1.000000


In [18]:
variable = 'Moving Defensive Before Snap'

q1_dist = thru_snap[variable].quantile(0.25)
q3_dist = thru_snap[variable].quantile(0.75)

dist_low = thru_snap[thru_snap[variable] <= q1_dist]
dist_mid = thru_snap[(thru_snap[variable] > q1_dist) & 
                        (thru_snap[variable] <= q3_dist)]
dist_high = thru_snap[thru_snap[variable] > q3_dist]

def print_group_stats(group, group_name):
    epa_mean = group["expectedPointsAdded"].mean()
    yards_mean = group['yardsGained'].mean()
    big_y_plays = group['bigYardPlay'].mean()
    big_epa_plays = group['bigEpaPlay'].mean()
    print(f"{group_name} — EPA Mean: {epa_mean:.3f}, Yards Gained Mean: {yards_mean:.2f}, BigYards: {big_y_plays:.2f}, BigEPA: {big_epa_plays:.2f}")

print_group_stats(dist_low, "variable < 25%")
print_group_stats(dist_mid, "variable 25%-75%")
print_group_stats(dist_high, "variable > 75%")

variable < 25% — EPA Mean: -0.023, Yards Gained Mean: 5.49, BigYards: 0.10, BigEPA: 0.11
variable 25%-75% — EPA Mean: nan, Yards Gained Mean: nan, BigYards: nan, BigEPA: nan
variable > 75% — EPA Mean: -0.049, Yards Gained Mean: 5.35, BigYards: 0.10, BigEPA: 0.11


##### There are a few issues with this function. You look at velocity 20 frames before and 20 frames after. A player will always be moving after snap. So your technically saying any player that moves within 20 frames before snap is moving THRU snap. You need to look at each frame before the snap and each frame after and make sure each frame is above a certain velocity threshold. You also didn't check for every defender. I'm modifying your function so it checks that the 10 frames before the snap and 10 frames after the snap are all above a velocity threshold of 1. So any defender moving at a velocity > 1 throughout the snap is marked.

In [19]:
games = pd.read_csv("~/nfl-data-bowl/Sample_Data/Raw/games.csv")
plays = pd.read_csv("~/nfl-data-bowl/Sample_Data/Raw/plays.csv")
players = pd.read_csv("~/nfl-data-bowl/Sample_Data/Raw/players.csv")
player_play = pd.read_csv("~/nfl-data-bowl/Sample_Data/Raw/player_play.csv")

weeks = {}
for i in range(1, 10):
    file_path = f"~/nfl-data-bowl/Sample_Data/Raw/tracking_week_{i}.csv" 
    weeks[i] = pd.read_csv(file_path)

KeyboardInterrupt: 

In [ ]:
def get_defenders(df, gameID, playID, team):
    """
    Get defenders for a specific game and play.

    Parameters:
    df (pd.DataFrame): DataFrame containing player_play data.
    gameID (int): The game ID.
    playID (int): The play ID.
    team (str): The offensive team abbreviation.

    Returns:
    list: List of defender NFL IDs.
    """
    filtered_df = df[(df['gameId'] == gameID) & 
                     (df['playId'] == playID) & 
                     (df['teamAbbr'] != team)]
    
    return filtered_df['nflId'].tolist()

In [ ]:
week1 = pd.read_csv("~/nfl-data-bowl/Sample_Data/Raw/tracking_week_1.csv" )

In [ ]:
def analyze_defensive_movement_through_snap(game_id, play_id, tracking_data, plays, player_play, frame_window=5, speed_threshold=1.0):
    """
    Detects defenders moving through the snap based on team affiliation and speed thresholds.
    Also calculates average velocity at SNAP and maximum speed across the window for those defenders.

    Args:
        game_id (int): Game ID.
        play_id (int): Play ID.
        tracking_data (pd.DataFrame): Tracking data.
        plays (pd.DataFrame): Plays data (to get defensive team).
        player_play (pd.DataFrame): Player play data (to get teamAbbr per player).
        frame_window (int): Number of frames before and after SNAP.
        speed_threshold (float): Minimum speed at each frame.

    Returns:
        tuple: (number_of_defenders_moving, average_snap_speed_of_movers, max_speed_of_movers)
    """

    # Get offensive and defensive teams
    play_info = plays[(plays['gameId'] == game_id) & (plays['playId'] == play_id)]
    if play_info.empty:
        return None, None, None

    offensive_team = play_info['possessionTeam'].values[0]
    defensive_team = play_info['defensiveTeam'].values[0]

    # Get defender NFL IDs
    defenders = get_defenders(player_play, game_id, play_id, offensive_team)

    # Get play tracking data
    play_data = tracking_data[(tracking_data['gameId'] == game_id) & (tracking_data['playId'] == play_id)]
    if play_data.empty:
        return None, None, None

    # Find SNAP frame
    snap_frame = play_data[play_data['frameType'] == 'SNAP']['frameId'].min()
    if pd.isna(snap_frame):
        return None, None, None

    # Define frames to check
    frames_to_check = list(range(snap_frame - frame_window, snap_frame + frame_window + 1))

    # Filter play data to only defenders and frames of interest
    defender_data = play_data[(play_data['nflId'].isin(defenders)) & (play_data['frameId'].isin(frames_to_check))]

    if defender_data.empty:
        return 0, 0.0, 0.0

    # Group by defender and check speed at all frames
    defenders_moving_through_snap = []
    for nfl_id, player_frames in defender_data.groupby('nflId'):
        if all(player_frames['s'] > speed_threshold):
            defenders_moving_through_snap.append(nfl_id)

    num_movers = len(defenders_moving_through_snap)

    if num_movers == 0:
        return 0, 0.0, 0.0

    # Now get speed at exactly SNAP frame for those defenders
    snap_data = play_data[(play_data['frameId'] == snap_frame) & (play_data['nflId'].isin(defenders_moving_through_snap))]
    average_snap_speed = snap_data['s'].mean()

    # Also get maximum speed across the window for those defenders
    defender_window_data = defender_data[defender_data['nflId'].isin(defenders_moving_through_snap)]
    max_speed = defender_window_data['s'].max()

    return num_movers, average_snap_speed, max_speed


value1, value2, value3 = analyze_defensive_movement_through_snap(2022090800, 2979, week1, plays, player_play, frame_window=5, speed_threshold=1.5)
print(value1, value2, value3)

1 2.28 3.24


In [ ]:
games = pd.read_csv("~/nfl-data-bowl/Sample_Data/Raw/games.csv")
plays = pd.read_csv("~/nfl-data-bowl/Sample_Data/Raw/plays.csv")
players = pd.read_csv("~/nfl-data-bowl/Sample_Data/Raw/players.csv")
player_play = pd.read_csv("~/nfl-data-bowl/Sample_Data/Raw/player_play.csv")

weeks = {}
for i in range(1, 10):
    file_path = f"~/nfl-data-bowl/Sample_Data/Raw/tracking_week_{i}.csv" 
    weeks[i] = pd.read_csv(file_path)

In [ ]:
import pandas as pd

def get_defenders(df, gameID, playID, team):
    filtered_df = df[(df['gameId'] == gameID) & 
                     (df['playId'] == playID) & 
                     (df['teamAbbr'] != team)]
    return filtered_df['nflId'].tolist()

def analyze_defensive_movement_through_snap(game_id, play_id, tracking_data, plays, player_play, frame_window=5, speed_threshold=1.0):
    play_info = plays[(plays['gameId'] == game_id) & (plays['playId'] == play_id)]
    if play_info.empty:
        return None, None, None

    offensive_team = play_info['possessionTeam'].values[0]

    defenders = get_defenders(player_play, game_id, play_id, offensive_team)

    play_data = tracking_data[(tracking_data['gameId'] == game_id) & (tracking_data['playId'] == play_id)]
    if play_data.empty:
        return None, None, None

    snap_frame = play_data[play_data['frameType'] == 'SNAP']['frameId'].min()
    if pd.isna(snap_frame):
        return None, None, None

    frames_to_check = list(range(snap_frame - 5, snap_frame + 6))

    defender_data = play_data[(play_data['nflId'].isin(defenders)) & (play_data['frameId'].isin(frames_to_check))]

    if defender_data.empty:
        return 0, 0.0, 0.0

    defenders_moving_through_snap = []
    for nfl_id, player_frames in defender_data.groupby('nflId'):
        if all(player_frames['s'] > speed_threshold):
            defenders_moving_through_snap.append(nfl_id)

    num_movers = len(defenders_moving_through_snap)

    if num_movers == 0:
        return 0, 0.0, 0.0

    snap_data = play_data[(play_data['frameId'] == snap_frame) & (play_data['nflId'].isin(defenders_moving_through_snap))]
    average_snap_speed = snap_data['s'].mean()

    defender_window_data = defender_data[defender_data['nflId'].isin(defenders_moving_through_snap)]
    max_speed = defender_window_data['s'].max()

    return num_movers, average_snap_speed, max_speed

def generate_defender_motion_table(plays, player_play, weeks, frame_window=5, speed_threshold=1.5):
    results = []

    for i in range(1, 10):
        tracking_data = weeks[i]

        for idx, play_row in plays.iterrows():
            game_id = play_row['gameId']
            play_id = play_row['playId']

            if game_id not in tracking_data['gameId'].values:
                continue

            num_defenders, avg_snap_speed, max_speed = analyze_defensive_movement_through_snap(
                game_id, play_id, tracking_data, plays, player_play,
                frame_window=frame_window, speed_threshold=speed_threshold
            )

            if num_defenders is not None:
                results.append({
                    'gameId': game_id,
                    'playId': play_id,
                    'num_defenders_moving': num_defenders,
                    'average_snap_speed': avg_snap_speed,
                    'max_speed_in_window': max_speed
                })

    return pd.DataFrame(results)


In [ ]:
import pandas as pd
from tqdm import tqdm  

game_to_week = {}
for i in range(1, 10):
    week_data = weeks[i]
    for game_id in week_data['gameId'].unique():
        game_to_week[game_id] = i

tracking_game_ids = set(game_to_week.keys())
plays = plays[plays['gameId'].isin(tracking_game_ids)]

results = []

for _, row in tqdm(plays.iterrows(), total=len(plays)):
    gameID = row["gameId"]
    playID = row["playId"]

    week_num = game_to_week.get(gameID)
    if week_num is None:
        continue

    tracking_week = weeks[week_num]

    num_defenders, avg_snap_speed, max_speed = analyze_defensive_movement_through_snap(
        gameID, playID, tracking_week, plays, player_play
    )

    if num_defenders is None:
        continue

    play_stats = {
        "gameID": gameID,
        "playID": playID,
        "num_defenders_moving": num_defenders,
        "average_snap_speed": avg_snap_speed,
        "max_speed_in_window": max_speed
    }

    results.append(play_stats)

final_df = pd.DataFrame(results)
final_df.to_csv("~/nfl-data-bowl/Sample_Data/Raw/motion_through_snap.csv", index=False)

print("✅ Processed plays saved to 'processed_defensive_motion_plays.csv'")


100%|██████████| 15873/15873 [11:27<00:00, 23.08it/s]


✅ Processed plays saved to 'processed_defensive_motion_plays.csv'


In [4]:
motion = pd.read_csv("~/nfl-data-bowl/Sample_Data/Raw/motion_through_snap.csv")

plays2 = plays[["gameId", "playId","expectedPointsAdded","yardsGained"]]

motion = motion.rename(columns={'gameID': 'gameId'})
motion = motion.rename(columns={'playID': 'playId'})

motion = plays2.merge(motion, on=['gameId', 'playId'], how='inner')

motion['bigYardPlay'] = (motion['yardsGained'] > 15).astype(int)
motion['bigEpaPlay'] = (motion['expectedPointsAdded'] > 1.5).astype(int)

motion.describe()

,gameId,playId,expectedPointsAdded,yardsGained,num_defenders_moving,average_snap_speed,max_speed_in_window,bigYardPlay,bigEpaPlay
count,1.584600e+04,15846.000000,15846.000000,15846.000000,15846.000000,15846.000000,15846.000000,15846.000000,15846.000000
mean,2.022099e+09,2019.265998,-0.029166,5.460305,0.746056,1.066121,1.428220,0.099016,0.110249
std,6.008236e+03,1179.807629,1.397381,8.812619,0.972520,1.318990,1.752148,0.298692,0.313209
min,2.022091e+09,54.000000,-13.023600,-68.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.022092e+09,994.000000,-0.645547,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.022101e+09,2011.500000,-0.178552,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.022102e+09,3017.000000,0.697921,8.000000,1.000000,1.890000,2.570000,0.000000,0.000000
max,2.022111e+09,5120.000000,8.698986,98.000000,11.000000,7.710000,8.230000,1.000000,1.000000


##### When no defenders are moving the average epa is -0.033, yards 5.4, big_yd = .099, big_epa = 0.11. When defenders are moving -0.025 epa, yards 5.5, big_yd = .099, big_epa = 0.107.

In [5]:
no_motion = motion[motion["num_defenders_moving"] == 0]

only_motion = motion[motion["num_defenders_moving"] > 0]

#no_motion.describe()
only_motion.describe()

,gameId,playId,expectedPointsAdded,yardsGained,num_defenders_moving,average_snap_speed,max_speed_in_window,bigYardPlay,bigEpaPlay
count,7.625000e+03,7625.000000,7625.000000,7625.000000,7625.000000,7625.000000,7625.000000,7625.000000,7625.000000
mean,2.022099e+09,1999.604721,-0.024811,5.530361,1.550426,2.215574,2.968076,0.099016,0.107410
std,6.003096e+03,1173.830087,1.381114,8.626757,0.847568,1.033765,1.345152,0.298704,0.309654
min,2.022091e+09,55.000000,-13.023600,-30.000000,1.000000,1.010000,1.080000,0.000000,0.000000
25%,2.022092e+09,983.000000,-0.637169,0.000000,1.000000,1.510000,1.970000,0.000000,0.000000
50%,2.022101e+09,1999.000000,-0.159725,3.000000,1.000000,1.936667,2.630000,0.000000,0.000000
75%,2.022102e+09,2993.000000,0.717727,8.000000,2.000000,2.560000,3.570000,0.000000,0.000000
max,2.022111e+09,5120.000000,6.851195,75.000000,11.000000,7.710000,8.230000,1.000000,1.000000


##### Checking num_defenders_moving

In [6]:
motion = only_motion
variable = 'num_defenders_moving'

q1_dist = motion[variable].quantile(0.25)
q3_dist = motion[variable].quantile(0.75)

dist_low = motion[motion[variable] <= q1_dist]
dist_mid = motion[(motion[variable] > q1_dist) & 
                        (motion[variable] <= q3_dist)]
dist_high = motion[motion[variable] > q3_dist]

def print_group_stats(group, group_name):
    epa_mean = group["expectedPointsAdded"].mean()
    yards_mean = group['yardsGained'].mean()
    big_y_plays = group['bigYardPlay'].mean()
    big_epa_plays = group['bigEpaPlay'].mean()
    print(f"{group_name} — EPA Mean: {epa_mean:.3f}, Yards Gained Mean: {yards_mean:.2f}, BigYards: {big_y_plays:.2f}, BigEPA: {big_epa_plays:.2f}")

print_group_stats(dist_low, "variable < 25%")
print_group_stats(dist_mid, "variable 25%-75%")
print_group_stats(dist_high, "variable > 75%")

top_20 = motion.nlargest(100, variable)
bottom_50 = motion.nsmallest(100, variable)

print_group_stats(bottom_50 , "bottom50")
print_group_stats(top_20, "top50")

variable < 25% — EPA Mean: -0.018, Yards Gained Mean: 5.54, BigYards: 0.10, BigEPA: 0.11
variable 25%-75% — EPA Mean: -0.035, Yards Gained Mean: 5.42, BigYards: 0.10, BigEPA: 0.10
variable > 75% — EPA Mean: -0.039, Yards Gained Mean: 5.74, BigYards: 0.10, BigEPA: 0.09
bottom50 — EPA Mean: -0.009, Yards Gained Mean: 5.52, BigYards: 0.09, BigEPA: 0.12
top50 — EPA Mean: -0.030, Yards Gained Mean: 6.04, BigYards: 0.11, BigEPA: 0.07


##### Checking average_snap_speed

In [23]:
variable = 'average_snap_speed'

q1_dist = motion[variable].quantile(0.25)
q3_dist = motion[variable].quantile(0.75)

dist_low = motion[motion[variable] <= q1_dist]
dist_mid = motion[(motion[variable] > q1_dist) & 
                        (motion[variable] <= q3_dist)]
dist_high = motion[motion[variable] > q3_dist]

def print_group_stats(group, group_name):
    epa_mean = group["expectedPointsAdded"].mean()
    yards_mean = group['yardsGained'].mean()
    big_y_plays = group['bigYardPlay'].mean()
    big_epa_plays = group['bigEpaPlay'].mean()
    print(f"{group_name} — EPA Mean: {epa_mean:.3f}, Yards Gained Mean: {yards_mean:.2f}, BigYards: {big_y_plays:.2f}, BigEPA: {big_epa_plays:.2f}")

print_group_stats(dist_low, "variable < 25%")
print_group_stats(dist_mid, "variable 25%-75%")
print_group_stats(dist_high, "variable > 75%")
top_20 = motion.nlargest(50, variable)
bottom_50 = motion.nsmallest(50, variable)

print_group_stats(bottom_50 , "bottom50")
print_group_stats(top_20, "top50")

variable < 25% — EPA Mean: 0.007, Yards Gained Mean: 5.65, BigYards: 0.10, BigEPA: 0.12
variable 25%-75% — EPA Mean: -0.044, Yards Gained Mean: 5.58, BigYards: 0.10, BigEPA: 0.10
variable > 75% — EPA Mean: -0.018, Yards Gained Mean: 5.30, BigYards: 0.11, BigEPA: 0.11
bottom50 — EPA Mean: -0.325, Yards Gained Mean: 3.38, BigYards: 0.02, BigEPA: 0.04
top50 — EPA Mean: 0.188, Yards Gained Mean: 5.40, BigYards: 0.10, BigEPA: 0.08


##### Checking max_speed_in_window

In [ ]:
variable = 'max_speed_in_window'

q1_dist = motion[variable].quantile(0.25)
q3_dist = motion[variable].quantile(0.75)

dist_low = motion[motion[variable] <= q1_dist]
dist_mid = motion[(motion[variable] > q1_dist) & 
                        (motion[variable] <= q3_dist)]
dist_high = motion[motion[variable] > q3_dist]

def print_group_stats(group, group_name):
    epa_mean = group["expectedPointsAdded"].mean()
    yards_mean = group['yardsGained'].mean()
    big_y_plays = group['bigYardPlay'].mean()
    big_epa_plays = group['bigEpaPlay'].mean()
    print(f"{group_name} — EPA Mean: {epa_mean:.3f}, Yards Gained Mean: {yards_mean:.2f}, BigYards: {big_y_plays:.2f}, BigEPA: {big_epa_plays:.2f}")

print_group_stats(dist_low, "variable < 25%")
print_group_stats(dist_mid, "variable 25%-75%")
print_group_stats(dist_high, "variable > 75%")

top_20 = motion.nlargest(100, variable)
bottom_50 = motion.nsmallest(100, variable)

print_group_stats(bottom_50 , "bottom100")
print_group_stats(top_20, "top100")

variable < 25% — EPA Mean: 0.017, Yards Gained Mean: 5.67, BigYards: 0.10, BigEPA: 0.12
variable 25%-75% — EPA Mean: -0.054, Yards Gained Mean: 5.53, BigYards: 0.10, BigEPA: 0.10
variable > 75% — EPA Mean: -0.008, Yards Gained Mean: 5.39, BigYards: 0.11, BigEPA: 0.11
bottom50 — EPA Mean: -0.198, Yards Gained Mean: 4.64, BigYards: 0.09, BigEPA: 0.08
top50 — EPA Mean: -0.182, Yards Gained Mean: 4.41, BigYards: 0.08, BigEPA: 0.08
